# Login 

In [13]:
import requests
import spacy
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from newspaper import Article, Config, ArticleException
from tqdm import tqdm
from bs4 import BeautifulSoup
from robin_stocks import robinhood as r
# ArticleException
from newspaper import Article, Config, ArticleException
from colorama import Fore, Back, Style
import json
import time
from ratelimit import limits, sleep_and_retry
import newspaper
import random
import fuzzywuzzy
from fuzzywuzzy import fuzz
!pip install yfinance --upgrade --no-cache-dir
import yfinance


# Load secrets
with open('secrets.json') as f:
    secrets = json.load(f)

# Robinhood login
username = secrets['username']
password = secrets['password']
login = r.login(username, password)

# Initialize NLP tools
nltk.download('vader_lexicon', quiet=True)
sia = SentimentIntensityAnalyzer()
nlp = spacy.load('en_core_web_sm')


In [14]:
influence_dict = {}

# Functions

In [15]:

# Define a list of news sources
# news_sources = [
#     'https://www.drudge.com/',
#     'https://www.bloomberg.com/',
#     'https://www.marketwatch.com/',
#     'https://www.fool.com/',
#     'https://www.businessinsider.com/',
#     'https://www.axios.com/',
#     'https://www.politico.com/',
#     'https://www.theatlantic.com/',
#     'https://www.newyorker.com/',
#     'https://www.economist.com/',
#     'https://www.usnews.com/',
#     'https://www.nationalreview.com/',
#     'https://www.motherjones.com/',
#     'https://www.salon.com/',
#     'https://www.slate.com/',
#     'https://www.vox.com/',
#     'https://www.thehill.com/',
#     'https://www.vice.com/',
#     'https://www.buzzfeed.com/',
#     'https://www.dailykos.com/',
    
# ]
# crypto_news_sources = [
#     'https://cointelegraph.com/tags/robinhood',
#     'https://finance.yahoo.com/news/'
#     'https://finance.yahoo.com/cryptocurrencies/',
#     'https://www.coindesk.com/',
#     'https://cryptonews.com/',
#     'https://crypto.news/',
#     'https://www.cryptoglobe.com/latest/',
#     'https://www.cryptocraft.com/',
#     'https://www.cryptopolitan.com/',
#     'https://www.reuters.com/technology/cryptocurrency',
#     'https://www.coinspeaker.com/',
#     'https://cryptopotato.com/crypto-news/',
#     'https://www.cryptocointrade.com/',
#     'https://www.bloomberg.com/crypto'

# ]

news_sources = ['https://www.reddit.com/r/StockMarket/',
                'https://www.reddit.com/r/stocks/',
                'https://www.reddit.com/r/Stock_Picks/',
                'https://www.reddit.com/r/StocksAndTrading/',
                'https://www.reddit.com/r/StockMarketNews/',
                'https://www.reddit.com/r/cryptocurrency/',
                'https://www.reddit.com/r/CryptoCurrencyTrading/',
                'https://www.reddit.com/r/CryptoMarkets/', 
                'https://www.reddit.com/r/CryptoCurrency/',
                'https://www.reddit.com/r/Crypto'
]

news_articles = []
cryptocurrencies = [
    "Aave", "AAVE",
    "Bitcoin", "BTC",
    "Ethereum", "ETH",
    "Ethereum Classic", "ETC",
    "Dogecoin", "DOGE",
    "Litecoin", "LTC",
    "Bitcoin Cash", "BCH",
    "Bitcoin SV", "BSV",
    "USD Coin", "USDC",
    "Cardano", "ADA",
    "Solana", "SOL",
    "Polygon", "MATIC",
    "Shiba Inu", "SHIB",
    "Avalanche", "AVAX",
    "Uniswap", "UNI",
    "Chainlink", "LINK",
    "Steller Lumens", "XLM",
    "Compound", "COMP",
    "Tezos", "XTZ"
    ]
def get_company_names(text):
    doc = nlp(text)
    # company_names = [ent.text for ent in doc.ents if ent.label_ == 'ORG']
    coin_names = [
    "Aave", "AAVE",
    "Bitcoin", "BTC",
    "Ethereum", "ETH",
    "Ethereum Classic", "ETC",
    "Dogecoin", "DOGE",
    "Litecoin", "LTC",
    "Bitcoin Cash", "BCH",
    "Bitcoin SV", "BSV",
    "USD Coin", "USDC",
    "Cardano", "ADA",
    "Solana", "SOL",
    "Polygon", "MATIC",
    "Shiba Inu", "SHIB",
    "Avalanche", "AVAX",
    "Uniswap", "UNI",
    "Chainlink", "LINK",
    "Steller Lumens", "XLM",
    "Compound", "COMP",
    "Tezos", "XTZ"
    ]
    cryptocurrencies = coin_names
    company_names = [] # list of company names
    crypto_names_found = [] # list of crypto names found in the article
    #todo = the company names will come into play when trading stocks during trading hours and will be used to determine which stocks to trade
    # check if it is during trading hours markets are open from 9:30am to 4:00pm
    #print(f' | {Fore.YELLOW} Current time: {time.localtime().tm_hour}{Style.RESET_ALL}, the markets are open from 9:30am to 4:00pm')
    if 9 < time.localtime().tm_hour < 16: # if it is during trading hours
        # Use Spacy's NER to identify company names
        company_names = [ent.text for ent in doc.ents if ent.label_ == 'ORG']
        # add variations of company names
        company_names += [name.lower() for name in company_names if len(name) > 3]
        company_names += [name.upper() for name in company_names if len(name) > 3]
        company_names += [name.replace(' ', '') for name in company_names]
        company_names += [name.replace(' ', '-') for name in company_names]
        company_names += [name.replace(' ', '_') for name in company_names]
        # remove duplicates
        company_names = list(set(company_names))
    else: # if it is not during trading hours
        # only use crypto_news_sources

        # Use Spacy's NER to identify Cryptocurrency names
        crypto_names_found = [ent.text for ent in doc.ents if ent.text in coin_names]
        # add variations of coin names
        crypto_names_found += [name.lower() for name in crypto_names_found if len(name) > 3]
        if len(crypto_names_found) > 0:
            print(f' | {Fore.YELLOW} Crypto names found: {crypto_names_found}{Style.RESET_ALL}')
    # append the crypto names to the company names
    company_names += crypto_names_found
    # remove duplicates
    company_names = list(set(company_names))

    return company_names

def calculate_sentiment(text):
    sentiment = sia.polarity_scores(text)
    sentiment_score = sentiment['compound']
    return sentiment_score

from fuzzywuzzy import fuzz

def levenshtein_similarity_check(new_article, articles_list):
    # make the articles_list object iterable
    print(type(articles_list))
    articles_list = list(articles_list)
    # iterate through the list of articles
    for article in articles_list:
        # compare new article with article
        if isinstance(article, Article):
            article = article.title # if the article is an Article object, get the title
        elif isinstance(article, str):
            article = article # if the article is a string, keep it as a string
        # calculate the similarity between the new article and the article
        similarity = fuzz.ratio(new_article, article)
        # if the similarity is greater than 50%
        if similarity > 50:
            # return True
            return True
    # if the similarity is less than 50%
    # return False
    return False

def get_crypto_names(article_text):
    # Return a list of cryptocurrency names that appear in the article text.
    return [crypto for crypto in cryptocurrencies if crypto in article_text]

sentiment_analyzer = SentimentIntensityAnalyzer()

def calculate_sentiment_of_sentence_containing_crypto(article_text):
    # Split the article into sentences, calculate sentiment for each sentence that contains a cryptocurrency,
    # and return the average sentiment score.
    sentences = article_text.split('. ')
    sentiment_scores = []
    for sentence in sentences:
        for crypto in cryptocurrencies:
            if crypto in sentence:
                sentiment_score = sentiment_analyzer.polarity_scores(sentence)["compound"]
                sentiment_scores.append(sentiment_score)
    return sum(sentiment_scores) / len(sentiment_scores) if sentiment_scores else 0
import re
def refine_list_of_articles(articles_list):
    # I only want articles about cryptocurrencies or companies
    # I don't want articles that are too similar to each other
    # I don't want articles that are too short
    # use NER to identify company names
    # use established list of cryptocurrency names
    # use fuzzywuzzy to check for similarity
    # use spacy to check for length
    article_objects = articles_list # list of article objects
    refined_articles_list = []
    for article in articles_list:
        if len(str(str(article.text))) > 1000:
            if not levenshtein_similarity_check(str(str(article.text)), refined_articles_list):
                refined_articles_list.append(article)
        # using NER
        company_names = get_company_names(str(article.text))
        if len(company_names) > 0:
            refined_articles_list.append(article)
        # using cryptocurrency names
        crypto_names = get_crypto_names(str(article.text))
        if len(crypto_names) > 0:
            refined_articles_list.append(article)
        if type(article.title) == str:
            # remove whitespace and extra spaces
            title = article.title.strip()
            title = re.sub(' +', ' ', title)
            article.title = title
            if len(str(title)) > 0:
                refined_articles_list.append(article)
                print(f' | {Fore.YELLOW} {title}{Style.RESET_ALL}')
        # if we added this article then add the object to the list of article objects
        if article in refined_articles_list:
            if isinstance(article, Article):
                article_objects.append(article)
            else:
                print(f'Error is happening on line 204')
    return refined_articles_list, article_objects


def get_news_sentiment(site_url, articles_list=[]):
    global influence_dict
    if 9 < time.localtime().tm_hour < 16: # if it is during trading hours
        articles_list = news_sources + crypto_news_sources
    else:
        articles_list = crypto_news_sources
    print(Fore.YELLOW + f"Scanning {site_url} for news articles...")
    paper = newspaper.build(site_url)
    print(Fore.GREEN + 'we have ' + str(len(paper.articles)) + ' articles' + Style.RESET_ALL)
    sentiment_scores = {}
    article_importances = {}
    
    print('selecting only ones that contain names of cryptocurrencies or companies')
    articles_list, article_objects = refine_list_of_articles(paper.articles)
    
    # now remove duplicate articles (from both lists) by checking article object.title values.
    without_duplicates = []
    article_objects_without_duplicates = []
    index_ = 0
    for article in articles_list:
        if article.title not in [a.title for a in without_duplicates]:
            without_duplicates.append(article)
            article_objects_without_duplicates.append(article_objects[index_])
        index_ += 1

    for article in tqdm(articles_list):
        if isinstance(article, Article):
            article_title_str = str(article.title)
            article_url_str = str(article.url)
            try:
                article.download()
                article.parse()
                tqdm.write(str(article.title))
                article_text = str(article.text)
            except Exception:
                pass
        elif isinstance(article, str):
            article_title_str = article
            article_url_str = ''
            article_text = article
        elif isinstance(article, dict):
            article_title_str = article['title']
            article_url_str = article['url']
            article_text = article['text']
        else:
            print('article is not a string or a dictionary')
            continue
        try:
            article.download()
            article.parse()
            tqdm.write(str(article.title))
            article_text = str(article.text)
        except Exception:
            print('Error: Could not download article')
            if levenshtein_similarity_check(str(article.title), articles_list):
                print(Fore.RED + 'article title is similar to another article, skipping')
                importance_factor = 1.25
            else:
                importance_factor = 1
                articles_list.append({'title': article.title, 'url': article.url,'importance_factor': importance_factor})
            continue
        # check this article against the other articles in the list
        #!sims = [levenshtein_similarity_check(article_text, articles_list) for article_text in articles_list]
        # let's take sims and just set it to False for now while we figure out how to make it work
        sims = [False] #todo -- change this back to sims = [levenshtein_similarity_check(article_text, articles_list) for article_text in articles_list]
        if any(sims):
            print(Fore.RED + 'article is similar to another article, skipping')
            importance_factor = 1.25
            article_importances[article_title_str] = importance_factor
        else:
            importance_factor = 1
            article_importances[article_title_str] = importance_factor
            sentiment_scores[article_title_str] = calculate_sentiment_of_sentence_containing_crypto(article_text)
            # importance_factor should be tracked in the articles_list
            articles_list.append({'title': article.title, 'url': article.url,'importance_factor': importance_factor})
    return sentiment_scores, articles_list  


def generate_trades(sentiment_scores):
    trades = []
    for company, scores in sentiment_scores.items():
        avg_sentiment_score = sum(scores) / len(scores)
        print(Fore.GREEN + f"{company} has an average sentiment score of {avg_sentiment_score}, evaluating trade...", Style.RESET_ALL)
        if avg_sentiment_score > 0.2:
            print(Fore.GREEN + f"Considering buying {company}, average sentiment score: {avg_sentiment_score}", Style.RESET_ALL)
            trades.append((company, 'BUY'))
        elif avg_sentiment_score < -0.2:
            trades.append((company, 'SELL'))
    return trades

def getTicker(company_name):
    """
    The getTicker function takes in a company name and returns the ticker symbol for that company.
        It does this by using the Yahoo Finance API to search for a given company, then returning 
        the first result's ticker symbol.
    
    :param company_name: Search for the company's ticker symbol
    :return: The company's ticker symbol
    :credit: bruhbruhroblox on GitHub
    :doc-author: Trelent
    """
    yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    params = {"q": company_name, "quotes_count": 1, "country": "United States"}

    res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
    data = res.json()

    company_code = data['quotes'][0]['symbol']
    return company_code

def trader(ticker, action, commodity_type='stock', signal_strength=0):
    """
    The trader function takes in a ticker, an action, and a signal strength.
    The function then checks the current price of the stock or crypto and compares it to your buying power.
    If you have enough money to buy at least one share of the stock or crypto, it will execute that order. 
    Otherwise, if you already own shares of this asset but are not currently holding any other assets (i.e., 
    you have no open orders), then it will sell all your shares for whatever they're worth now.
    
    :param ticker: Specify the stock or crypto symbol
    :param action: Determine whether to buy, sell or hold a stock
    :param commodity_type: Determine whether the stock is a stock or crypto
    :param signal_strength: Determine the amount of shares to buy
    :return: A dictionary with the following keys:
    
    variables internally:
    - purchase_limit: maximum amount for a buy operation
    - current_price: current price of the stock or crypto
    - profile: the user's profile
    - buying_power: the user's buying power
    - ticker: the stock or crypto symbol
    - action: whether to buy, sell or hold a stock
    - commodity_type: whether the stock is a stock or crypto
    - signal_strength: the amount of shares to buy
    - company_name: the name of the company
    - company_code: the company's ticker symbol
    - yfinance: the Yahoo Finance API
    
    :doc-author: Trelent
    """
    
    profile = r.profiles.load_account_profile()
    buying_power = float(profile['buying_power'])
    # purchase_limit = buying_power * 0.01  # maximum amount for a buy operation
    #todo -- this is temp
    purchase_limit = 5.00 # maximum amount for a buy operation in USD
    # if the ticker has the format "BTC-USD", then it's a crypto and we need to remove the "-USD" part
    if '-' in ticker:
        ticker = ticker.split('-')[0]
    # get the current price of the stock or crypto
    try:
        latest_price = r.stocks.get_latest_price(ticker)[0]  # current price of the stock or crypto
    except TypeError: 
        latest_price = r.crypto.get_crypto_quote(ticker)['mark_price']  # current price of the stock or crypto
    except Exception as e:
        print(e)
        return
    latest_price = float(latest_price)  # current price of the stock or crypto
    current_price = float(latest_price)  # current price of the stock or crypto
    # also convert the buying power to a float
    buying_power = float(buying_power)
    # check if the user has enough money to buy at least one share of the stock or crypto
    if buying_power < purchase_limit:
        print(f'You do not have enough buying power to buy {ticker}')
        return
    # check if the user already owns shares of this asset
    if ticker in r.build_holdings():
        # check if the user is currently holding any other assets (i.e., they have open orders)
        if len(r.build_holdings()) > 1:
            print(f'You already own shares of {ticker} and are holding other assets. Not buying or selling.')
            return
        # if the user is not holding any other assets, sell all their shares for whatever they're worth now
        else:
            volume = float(r.build_holdings()[ticker]['quantity'])
            r.orders.order_sell_fractional_by_quantity(ticker, volume, stop_price=current_price * 0.99)
            return
    # if the user does not already own shares of this asset, check if they have any open orders
    elif len(r.build_holdings()) > 0:
        print(f'You do not own shares of {ticker} and are holding other assets. Not buying or selling.')
        return
    # if the user does not already own shares of this asset and is not holding any other assets, buy shares of the asset
    else:
        try:
            # buy $1.00 worth of the stock or crypto
            if action == 'BUY':
                if commodity_type == 'stock':
                    if current_price < 5 and purchase_limit >= 5:
                        r.orders.order_buy_fractional_by_price(ticker, 5)
                    elif purchase_limit >= current_price:
                        r.orders.order_buy_fractional_by_price(ticker, purchase_limit)
                elif commodity_type == 'crypto':
                    if purchase_limit >= current_price:
                        r.orders.order_buy_crypto_by_price(ticker, purchase_limit)
                else:
                    raise Exception('Invalid commodity type')
            # sell all shares of the stock or crypto
            elif action == 'SELL':
                volume = float(r.build_holdings()[ticker]['quantity'])
                r.orders.order_sell_fractional_by_quantity(ticker, volume, stop_price=current_price * 0.99)
            # stop loss order sell
            elif action == 'STOP_LOSS_ORDER_SELL':
                volume = float(r.build_holdings()[ticker]['quantity'])
                r.orders.order_sell_fractional_by_quantity(ticker, volume, stop_price=current_price * 0.99)
            else:
                raise Exception('Invalid action')
        except Exception as e:
            print(e)
            return
    
    if action == 'BUY':
        if commodity_type == 'stock':
            if current_price < 5 and purchase_limit >= 5:
                r.orders.order_buy_fractional_by_price(ticker, 5)
            elif purchase_limit >= current_price:
                r.orders.order_buy_fractional_by_price(ticker, purchase_limit)
        elif commodity_type == 'crypto':
            if purchase_limit >= current_price:
                r.orders.order_buy_crypto_by_price(ticker, purchase_limit)
        else:
            raise Exception('Invalid commodity type')
            
    elif action == 'SELL':
        volume = float(r.build_holdings()[ticker]['quantity'])
        r.orders.order_sell_fractional_by_quantity(ticker, volume, stop_price=current_price * 0.99)
        
    elif action == 'STOP_LOSS_ORDER_SELL':
        volume = float(r.build_holdings()[ticker]['quantity'])
        r.orders.order_sell_stop_loss(symbol=ticker, quantity=volume, stop_price=current_price * 0.99)
        
    elif action == 'HOLD':
        pass

    else:
        raise Exception('Invalid action')

def getTicker(company_name):
    company_ticker=''
    try:
        company_ticker = yfinance.Ticker(company_name).info['symbol']
    except Exception as e:
        company_ticker = yfinance.Ticker(company_name + '.PA').info['symbol']
    # if there is no ticker for this company, then we can't trade it so return None
    if company_ticker == 'N/A' or company_ticker == '':
        return None

def execute_trades(trades):
    profile = r.profiles.load_account_profile()
    holdings = r.build_holdings()
    owned_stocks = list(holdings.keys())
    buying_power = float(profile['buying_power'])
    for trade in tqdm(trades):
        company, action = trade
        try:
            company_ticker = getTicker(company)
        except Exception as e:
            #print(Fore.RED + f"Could not find ticker for {company}, skipping trade", Style.RESET_ALL)
            continue # we couldn't find the ticker for this company, so we can't trade it
        if company_ticker not in owned_stocks and action == 'SELL':
            continue # we can't sell a stock we don't own
        position_size = min(buying_power * 0.01, buying_power)
        if action == 'BUY':
            print(f"Buying {position_size} shares of {company}")
            time.sleep(4)
            trader(company_ticker, action='BUY') # buy 1% of our buying power worth of this stock
            time.sleep(random.randint(1,4))
            # also place a sell stop loss order to trigger if the price raises 1% of the current price (to protect against a crash)
            print('+ (additional order)\n\tSELLING {position_size} shares of ' + Fore.BLUE + ' {company} and setting stop loss at {1.01*position_size} ' + Style.RESET_ALL)
            r.orders.order_sell_stop_loss(symbol=company,quantity=position_size,stopPrice = 1.01*position_size, timeInForce='gfc') # sell 95% of our position, with a stop loss of 95% of our position, good for the day
        elif action == 'SELL':
            if company not in owned_stocks:
                print(Fore.RED + f"Cannot sell {company}, we don't own it!", Style.RESET_ALL)
                continue
            # when selling, we want to sell 95% of our position
            print('SELLING {position_size} shares of ' + Fore.BLUE + ' {company} and setting stop loss at {0.95*position_size} ' + Style.RESET_ALL)
            time.sleep(4)
            r.orders.order_sell_stop_loss(symbol=company,quantity=0.95*position_size,stopPrice = 0.95*position_size, timeInForce='gfd') # sell 95% of our position, with a stop loss of 95% of our position, good for the day

def prune_articles(article_objects, articles):
    # remove articles with the titles 'Reply' or some number + 'comments'
    article_objects = [article for article in article_objects if 'Reply' not in article.title and 'comments' not in article.title]
    #? remove articles that have already been traded
    # article_objects = [article for article in article_objects if article.title not in articles]
    return article_objects


# Simple Mode


# Execution of the Functions

In [16]:
import time

In [17]:
# Main execution loop
def main():
    sentiment_scores, articles_list = get_news_sentiment(site_url)

    # articles_list is a list of dictionaries, each dictionary contains the title, url, and importance_factor of an article
    # importance factor - signifies the number of similar titles to this article, used to determine how important this article is
    # sentiment_scores is a dictionary, each key is a company name, each value is a list of sentiment scores for that company
    # save the articles_list to a file
    with open('articles_list.json', 'w') as f:
        json.dump(articles_list, f)
    if sentiment_scores:
        print(Fore.GREEN + f"Found sentiment for {site_url}, generating trades...")
        trades = generate_trades(sentiment_scores)
        execute_trades(trades)
    else:
        print(Fore.RED + f"Did not find sentiment for {site_url}")
# import time
while True:
    # shuffle the news sources so we don't hit the API rate limit
    random.shuffle(news_sources)
    # news_sources = ['https://www.reddit.com/r/StockMarket/']
    print('Starting loop')
    for site_url in news_sources:
        # use limits to avoid hitting the API rate limit
        main()
    time.sleep(60*30)


Starting loop
Scanning https://www.reddit.com/r/Crypto for news articles...
we have 2 articles
selecting only ones that contain names of cryptocurrencies or companies


0it [00:00, ?it/s]


Did not find sentiment for https://www.reddit.com/r/Crypto
Scanning https://www.reddit.com/r/stocks/ for news articles...
we have 0 articles
selecting only ones that contain names of cryptocurrencies or companies


0it [00:00, ?it/s]

Did not find sentiment for https://www.reddit.com/r/stocks/
Scanning https://www.reddit.com/r/Stock_Picks/ for news articles...


...

0it [00:00, ?it/s]

Did not find sentiment for https://www.reddit.com/r/StockMarketNews/
Scanning https://www.reddit.com/r/cryptocurrency/ for news articles...


we have 0 articles
selecting only ones that contain names of cryptocurrencies or companies


0it [00:00, ?it/s]

Did not find sentiment for https://www.reddit.com/r/cryptocurrency/
Scanning https://www.reddit.com/r/CryptoCurrencyTrading/ for news articles...
